# Object Detection Project (WIP)

dataset:  https://github.com/Shenggan/BCCD_Dataset

The purpose of this project is to train a RCNN that detects different
blood cell types in an image and create a bounding box around
the detected objects.

In [2]:
import numpy as np
import os
#import tensorflow as tf
import cv2
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt


     -------------------------------------- 117.2/117.2 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 948.0/948.0 kB 2.9 MB/s eta 0:00:00
     ---------------------------------------- 35.5/35.5 MB 3.7 MB/s eta 0:00:00
  Using cached opencv_python-4.6.0.66-cp36-abi3-win_amd64.whl (35.6 MB)
     ---------------------------------------- 1.3/1.3 MB 4.7 MB/s eta 0:00:00
  Created wheel for albumentations: filename=albumentations-0.4.6-py3-none-any.whl size=65162 sha256=7b81969b8695c004f269db82db495a3f272146322b7c70858b280f9abed6ad36
  Stored in directory: c:\users\kanen\appdata\local\pip\cache\wheels\cf\34\0f\cb2a5f93561a181a4bcc84847ad6aaceea8b5a3127469616cc
Successfully built albumentations
[WinError 2] The system cannot find the file specified: 'vision'
C:\Users\kanen\Desktop\Python\ML\Blood-Cell-Detection-Model


'git' is not recognized as an internal or external command,
operable program or batch file.


C:\Users\kanen\Desktop\Python\ML


'git' is not recognized as an internal or external command,
operable program or batch file.
'cp' is not recognized as an internal or external command,
operable program or batch file.
'cp' is not recognized as an internal or external command,
operable program or batch file.
'cp' is not recognized as an internal or external command,
operable program or batch file.
'cp' is not recognized as an internal or external command,
operable program or batch file.
'cp' is not recognized as an internal or external command,
operable program or batch file.


In [3]:

import torch
import torchvision
from torchvision import transforms as torchtrans  
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

#### Read data

In [105]:
df = pd.read_csv('test.csv')
print(df)

                  filename  cell_type  xmin  xmax  ymin  ymax
0     BloodImage_00000.jpg        WBC   260   491   177   376
1     BloodImage_00000.jpg        RBC    78   184   336   435
2     BloodImage_00000.jpg        RBC    63   169   237   336
3     BloodImage_00000.jpg        RBC   214   320   362   461
4     BloodImage_00000.jpg        RBC   414   506   352   445
...                    ...        ...   ...   ...   ...   ...
4883  BloodImage_00410.jpg  Platelets   239   291   275   321
4884  BloodImage_00410.jpg  Platelets   121   189   260   320
4885  BloodImage_00410.jpg  Platelets    57   104   119   167
4886  BloodImage_00410.jpg  Platelets     1    29   286   327
4887  BloodImage_00410.jpg        WBC   367   611   166   394

[4888 rows x 6 columns]


#### Data Preprocessing

In [106]:
class_names = [_, 'Platelets', 'RBC', 'WBC']

In [107]:
# Numerically encode data labels
le = preprocessing.LabelEncoder()
le.fit(class_names)
df['cell_type'] = le.transform(df['cell_type'])

In [108]:
# Normalize bounding box values
imgWidth = 640
imgHeight = 480
print(df)
df['xmin'] /= imgWidth
df['xmax'] /= imgWidth
df['ymin'] /= imgHeight
df['ymax'] /= imgHeight

                  filename  cell_type  xmin  xmax  ymin  ymax
0     BloodImage_00000.jpg          3   260   491   177   376
1     BloodImage_00000.jpg          2    78   184   336   435
2     BloodImage_00000.jpg          2    63   169   237   336
3     BloodImage_00000.jpg          2   214   320   362   461
4     BloodImage_00000.jpg          2   414   506   352   445
...                    ...        ...   ...   ...   ...   ...
4883  BloodImage_00410.jpg          1   239   291   275   321
4884  BloodImage_00410.jpg          1   121   189   260   320
4885  BloodImage_00410.jpg          1    57   104   119   167
4886  BloodImage_00410.jpg          1     1    29   286   327
4887  BloodImage_00410.jpg          3   367   611   166   394

[4888 rows x 6 columns]


In [109]:
print(df)

                  filename  cell_type      xmin      xmax      ymin      ymax
0     BloodImage_00000.jpg          3  0.406250  0.767188  0.368750  0.783333
1     BloodImage_00000.jpg          2  0.121875  0.287500  0.700000  0.906250
2     BloodImage_00000.jpg          2  0.098437  0.264062  0.493750  0.700000
3     BloodImage_00000.jpg          2  0.334375  0.500000  0.754167  0.960417
4     BloodImage_00000.jpg          2  0.646875  0.790625  0.733333  0.927083
...                    ...        ...       ...       ...       ...       ...
4883  BloodImage_00410.jpg          1  0.373437  0.454688  0.572917  0.668750
4884  BloodImage_00410.jpg          1  0.189062  0.295312  0.541667  0.666667
4885  BloodImage_00410.jpg          1  0.089063  0.162500  0.247917  0.347917
4886  BloodImage_00410.jpg          1  0.001563  0.045312  0.595833  0.681250
4887  BloodImage_00410.jpg          3  0.573438  0.954688  0.345833  0.820833

[4888 rows x 6 columns]


In [110]:
# Load in images

In [111]:
imgNames = pd.unique(df['filename'])
imgDirectory = ('BCCD\\JPEGImages\\')

imgs = np.zeros([len(imgNames), imgHeight, imgWidth, 3])
for idx, imgName in enumerate(imgNames):
    imgs[idx, :, :, :] = cv2.imread(imgDirectory + imgName)
    
print(np.shape(imgs))


(364, 480, 640, 3)


In [112]:
le = preprocessing.LabelEncoder()
le.fit(imgNames)
df['imgs_idx'] = le.transform(df['filename'])

In [113]:
print(df)

                  filename  cell_type      xmin      xmax      ymin      ymax  \
0     BloodImage_00000.jpg          3  0.406250  0.767188  0.368750  0.783333   
1     BloodImage_00000.jpg          2  0.121875  0.287500  0.700000  0.906250   
2     BloodImage_00000.jpg          2  0.098437  0.264062  0.493750  0.700000   
3     BloodImage_00000.jpg          2  0.334375  0.500000  0.754167  0.960417   
4     BloodImage_00000.jpg          2  0.646875  0.790625  0.733333  0.927083   
...                    ...        ...       ...       ...       ...       ...   
4883  BloodImage_00410.jpg          1  0.373437  0.454688  0.572917  0.668750   
4884  BloodImage_00410.jpg          1  0.189062  0.295312  0.541667  0.666667   
4885  BloodImage_00410.jpg          1  0.089063  0.162500  0.247917  0.347917   
4886  BloodImage_00410.jpg          1  0.001563  0.045312  0.595833  0.681250   
4887  BloodImage_00410.jpg          3  0.573438  0.954688  0.345833  0.820833   

      imgs_idx  
0         

#### Data Augmentation

In [114]:
# Flip images upside down
imgsFlipped = np.array(imgs, copy=True)  
print(np.shape(imgsFlipped))
for idx, img in enumerate(imgsFlipped):
    imgsFlipped[idx, :, :, :] = np.flipud(img)

(364, 480, 640, 3)


In [115]:
# Mirror images along y-axis
imgsMirrored = np.array(imgs, copy=True)
for idx, img in enumerate(imgsMirrored):
    imgsMirrored[idx, :, :, :] = np.fliplr(img)

In [116]:
imgShape = np.shape(imgs[0])

# Combine all images and augmented images into one numpy array
imgsAll = np.zeros([len(imgs)*3, imgShape[0], imgShape[1], imgShape[2]])
imgsAll[:len(imgs), :, :, :] = imgs
imgsAll[len(imgs):len(imgs)*2, :, :, :] = imgsFlipped
imgsAll[len(imgs)*2:len(imgs)*3, :, :, :] = imgsMirrored

print(np.shape(imgsAll))

(1092, 480, 640, 3)


In [117]:
df = df.drop(columns=['filename'])
print(df)

      cell_type      xmin      xmax      ymin      ymax  imgs_idx
0             3  0.406250  0.767188  0.368750  0.783333         0
1             2  0.121875  0.287500  0.700000  0.906250         0
2             2  0.098437  0.264062  0.493750  0.700000         0
3             2  0.334375  0.500000  0.754167  0.960417         0
4             2  0.646875  0.790625  0.733333  0.927083         0
...         ...       ...       ...       ...       ...       ...
4883          1  0.373437  0.454688  0.572917  0.668750       363
4884          1  0.189062  0.295312  0.541667  0.666667       363
4885          1  0.089063  0.162500  0.247917  0.347917       363
4886          1  0.001563  0.045312  0.595833  0.681250       363
4887          3  0.573438  0.954688  0.345833  0.820833       363

[4888 rows x 6 columns]


In [118]:
# Adjust bounding box to flip
dfFlipped = df.copy(deep=True)
dfFlipped['ymin'] = 1 - df['ymax']
dfFlipped['ymax'] = 1 - df['ymin']
dfFlipped['imgs_idx'] = df['imgs_idx'] + len(imgs)
print(dfFlipped)

      cell_type      xmin      xmax      ymin      ymax  imgs_idx
0             3  0.406250  0.767188  0.216667  0.631250       364
1             2  0.121875  0.287500  0.093750  0.300000       364
2             2  0.098437  0.264062  0.300000  0.506250       364
3             2  0.334375  0.500000  0.039583  0.245833       364
4             2  0.646875  0.790625  0.072917  0.266667       364
...         ...       ...       ...       ...       ...       ...
4883          1  0.373437  0.454688  0.331250  0.427083       727
4884          1  0.189062  0.295312  0.333333  0.458333       727
4885          1  0.089063  0.162500  0.652083  0.752083       727
4886          1  0.001563  0.045312  0.318750  0.404167       727
4887          3  0.573438  0.954688  0.179167  0.654167       727

[4888 rows x 6 columns]


In [119]:
# Adjust bounding box to mirroring
dfMirrored = df.copy(deep=True)
dfMirrored['xmin'] = 1 - df['xmax']
dfMirrored['xmax'] = 1 - df['xmin']
dfMirrored['imgs_idx'] = dfFlipped['imgs_idx'] + len(imgs)
print(dfMirrored)

      cell_type      xmin      xmax      ymin      ymax  imgs_idx
0             3  0.232812  0.593750  0.368750  0.783333       728
1             2  0.712500  0.878125  0.700000  0.906250       728
2             2  0.735938  0.901563  0.493750  0.700000       728
3             2  0.500000  0.665625  0.754167  0.960417       728
4             2  0.209375  0.353125  0.733333  0.927083       728
...         ...       ...       ...       ...       ...       ...
4883          1  0.545312  0.626563  0.572917  0.668750      1091
4884          1  0.704688  0.810937  0.541667  0.666667      1091
4885          1  0.837500  0.910937  0.247917  0.347917      1091
4886          1  0.954688  0.998437  0.595833  0.681250      1091
4887          3  0.045312  0.426562  0.345833  0.820833      1091

[4888 rows x 6 columns]


In [120]:
dfAll = pd.concat([df, dfFlipped, dfMirrored], ignore_index=True)
print(dfAll)
imgs = imgsAll
df = dfAll

       cell_type      xmin      xmax      ymin      ymax  imgs_idx
0              3  0.406250  0.767188  0.368750  0.783333         0
1              2  0.121875  0.287500  0.700000  0.906250         0
2              2  0.098437  0.264062  0.493750  0.700000         0
3              2  0.334375  0.500000  0.754167  0.960417         0
4              2  0.646875  0.790625  0.733333  0.927083         0
...          ...       ...       ...       ...       ...       ...
14659          1  0.545312  0.626563  0.572917  0.668750      1091
14660          1  0.704688  0.810937  0.541667  0.666667      1091
14661          1  0.837500  0.910937  0.247917  0.347917      1091
14662          1  0.954688  0.998437  0.595833  0.681250      1091
14663          3  0.045312  0.426562  0.345833  0.820833      1091

[14664 rows x 6 columns]


In [127]:
import matplotlib.patches as patches

# Function to show image with bounding boxes
def plot_boxes(img, boxes):
    imgHeight, imgWidth = np.shape(img)[0], np.shape(img)[1]
    
    fig, a = plt.subplots(1,1)
    fig.set_size_inches(5,5)
    a.imshow(img)
    
    #Draw red box for red blood cell, white for white blood cell, and blue for platelet
    for box in boxes:
        if box[4] == 1:
            width = (box[1]-box[0])*imgWidth
            height = (box[3]-box[2])*imgHeight
            rect = patches.Rectangle(
                (box[0]*imgWidth, box[2]*imgHeight),
                width, height,
                linewidth = 2,
                edgecolor = 'b',
                facecolor = 'none'
            )
        if box[4] == 2:
            width = (box[1]-box[0])*imgWidth
            height = (box[3]-box[2])*imgHeight
            rect = patches.Rectangle(
                (box[0]*imgWidth, box[2]*imgHeight),
                width, height,
                linewidth = 2,
                edgecolor = 'r',
                facecolor = 'none'
            )
        if box[4] == 3:
            width = (box[1]-box[0])*imgWidth
            height = (box[3]-box[2])*imgHeight
            rect = patches.Rectangle(
                (box[0]*imgWidth, box[2]*imgHeight),
                width, height,
                linewidth = 2,
                edgecolor = 'w',
                facecolor = 'none'
            )
    # Draw the bounding box on top of the image
        a.add_patch(rect)
    plt.show()

Demonstrated the data augmentation by showing the regular image and the flipped and mirrored versions.
The bounding box for the red blood cell is red, white for the white blood cell, and blue for platelets.

In [1]:
idx = 15
boxes = df.where(df['imgs_idx']==idx)[['xmin', 'xmax', 'ymin', 'ymax', 'cell_type']].dropna().to_numpy()
#print(boxes)
plot_boxes(cv2.cvtColor(imgsAll[idx].astype('uint8'), cv2.COLOR_BGR2RGB), boxes, )

boxes = df.where(df['imgs_idx']==idx+364)[['xmin', 'xmax', 'ymin', 'ymax', 'cell_type']].dropna().to_numpy()
plot_boxes(cv2.cvtColor(imgsAll[idx+364].astype('uint8'), cv2.COLOR_BGR2RGB), boxes)

boxes = df.where(df['imgs_idx']==idx+728)[['xmin', 'xmax', 'ymin', 'ymax', 'cell_type']].dropna().to_numpy()
plot_boxes(cv2.cvtColor(imgsAll[idx+728].astype('uint8'), cv2.COLOR_BGR2RGB), boxes)

NameError: name 'df' is not defined

### Dataset and Dataset loader class
Pytorch requires a class to process the data to the correct format and a dataset loader class which feeds the model the data for training and testing


In [ ]:
imgs = imgsAll
df = dfAll

# we create a Dataset class which has a __getitem__ function and a __len__ function
class BloodCellDataset(torch.utils.data.Dataset):

  def __init__(self, width, height, classes, transforms=None):
    self.transforms = transforms
    self.height = height
    self.width = width
    
    # sorting the images for consistency
    # To get images, the extension of the filename is checked to be jpg
    # classes: 0 index is reserved for background
    self.classes = classes

  def __getitem__(self, idx):
    # reading the images and converting them to correct size and color    
    img_rgb = cv2.cvtColor(imgs[idx], cv2.COLOR_BGR2RGB).astype(np.float32)
    img_res = cv2.resize(img_rgb, (self.width, self.height), cv2.INTER_AREA)
    # diving by 255
    img_res /= 255.0
    
    # annotation file
    annot_filename = img_name[:-4] + '.txt'
    annot_file_path = os.path.join(self.files_dir, annot_filename)
    
    boxes = df.where(df['imgs_idx']==idx)[
        ['xmin', 'xmax', 'ymin', 'ymax'].dropna().to_numpy()
    labels = df.where(df['imgs_idx']==idx)['cell_type'].dropna().to_numpy()
    
    # cv2 image gives size as height x width
    imageWidth = img.shape[1]
    imageHeight = img.shape[0]
    
    
    # convert boxes into a torch.Tensor
    boxes = torch.as_tensor(boxes, dtype=torch.float32)
    
    # getting the areas of the boxes
    area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])

    # suppose all instances are not crowd
    iscrowd = torch.zeros((boxes.shape[0],), dtype=torch.int64)
    
    labels = torch.as_tensor(labels, dtype=torch.int64)

    target = {}
    target["boxes"] = boxes
    target["labels"] = labels
    target["area"] = area
    target["iscrowd"] = iscrowd
    image_id = torch.tensor([idx])
    target["image_id"] = image_id

    if self.transforms:
      sample = self.transforms(image = img_res,
                                bboxes = target['boxes'],
                                labels = labels)
      img_res = sample['image']
      target['boxes'] = torch.Tensor(sample['bboxes'])
        
    return img_res, target

  def __len__(self):
    return len(self.imgs)


# check dataset
dataset = BloodCellDataset(files_dir, 224, 224)
print('Length of dataset:', len(dataset), '\n')

# getting the image and target for a test index.  Feel free to change the index.
img, target = dataset[78]
print('Image shape:', img.shape)
print('Label example:', target)

#### Transfer Learning

For this project I am using the SSD MobileNet V2 FPNLite 640x640 pre-trained model since
it is lightweight and the data input size is relatively close to the sizes of the images 
being used for this project

NameError: name 'KE' is not defined

NameError: name 'config_util' is not defined

<class 'tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject'>
